In [23]:
targets = [
    r"^아니다", 
    r"^없다", 
    r"^않다", 
    r"^안하다", 
    r" 아니다", 
    r" 없다", 
    r" 않다", 
    r" 안하다"
    ]

input_path = "./output/flattened_koksl2022_ann.json"


In [24]:
import json

with open(input_path, "r", encoding="utf-8") as file:
    records = json.loads(file.read())
    

In [25]:
from pathlib import Path

eaf_dir = Path("D:/koksl2022/EAF")
eaf_paths = eaf_dir.glob("*.eaf")

def path_mapper(path: Path):
    key = path.stem
    path_posix = path.as_posix()
    return (key, path_posix)

eaf_path_map = dict(path_mapper(p) for p in eaf_paths)


In [26]:
import re

def has_gloss(target, record):

    def get_glosses(record):
        ms_strong = (item["gloss_id"] for item in record["ms_strong"])
        ms_weak = (item["gloss_id"] for item in record["ms_weak"]) 
        nms = (item["gloss_id"] for item in record["nms"]) 
        return {*ms_strong, *ms_weak, *nms}

    glosses = get_glosses(record)
    matched = (bool(re.search(target, gl)) for gl in glosses)

    return True in matched


In [27]:
import os
import shutil

for target in targets:

    matched_record_ids = [rec["id"] for rec in records if has_gloss(target, rec)]

    out_dir = Path(f"./output/용례-{target}")

    if out_dir.exists():
        shutil.rmtree(out_dir)

    os.mkdir(out_dir)

    for i in matched_record_ids:
        shutil.copy(eaf_path_map[i], out_dir)
